In [46]:
import spacy # type: ignore
import pymupdf4llm # type: ignore
import os
from pathlib import Path
import json
from tqdm import tqdm # type: ignore
from openai import OpenAI
import os
from dotenv import load_dotenv
import pandas as pd
import requests
import uuid
import re

In [47]:
csv_file_path = '/Users/jamesmckinnon/Documents/re_analytics/software/backend/data/cre_listings/all_cre_listings.csv'
brochures_file_path = Path('/Volumes/James Mck/re_analysis/data/brochures/')
processed_brochures_file_path = Path('/Volumes/James Mck/re_analysis/data/processed_brochures/')

load_dotenv(dotenv_path="../../configs/.env")
OpenAI.api_key = os.getenv("OPENAI_API_KEY")

In [48]:
all_listings_df = pd.read_csv(csv_file_path)
brochure_urls = all_listings_df[all_listings_df['city'] == 'Edmonton']['brochure_urls'].dropna()

csv_file_path = '/Users/jamesmckinnon/Documents/re_analytics/software/backend/data/cre_listings/all_cre_listings.csv'
download_dir = '/Volumes/James Mck/re_analysis/data/brochures'

# Read the CSV file using pandas and extract brochure URLs
all_listings_df = pd.read_csv(csv_file_path)
all_listings_df = all_listings_df[all_listings_df['city'] == 'Edmonton'].dropna(subset=['brochure_urls'])
all_listings_df['uuid'] = [uuid.uuid4().hex  for _ in range(len(all_listings_df))]

In [49]:
input_dir = Path('/Volumes/James Mck/re_analysis/data/brochures/')
PDF_file_names = [f for f in os.listdir(input_dir) if not f.startswith("._") and f.endswith(".pdf")]
brochures_to_process = []

for file_name in PDF_file_names:
    file_name = file_name.replace('.pdf', '')
    matching_rows_df = all_listings_df[all_listings_df["brochure_urls"].str.contains(file_name, na=False)]

    for index, row in matching_rows_df.iterrows():
        temp_dict = {}
        temp_dict['uuid'] = row['uuid']
        temp_dict['file_name'] = file_name
        
        brochures_to_process.append(temp_dict)

In [50]:
client = OpenAI()
assistant = client.beta.assistants.create(
    name="Real Estate Data Extractor",
    instructions="You are an expert commercial real estate market analyst. Use your understanding of real estate to retrieve the information I need from the provided property brochures.",
    model="gpt-4o-mini-2024-07-18",
    tools=[{"type": "file_search"}]  # Enables file handling
)

In [51]:
# Create a vector store. This will contain embeddings of the document chunks once they are attached in the prompt.
vector_store = client.beta.vector_stores.create(name="CRE Brochures")

assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [ ]:
processed_brochures_list = []
output_json_file = f'processed_brochures_{len(brochures_to_process)}.json'

# Loop over file paths or brochure URLs within DAG
for brochure in brochures_to_process:
    uuid = brochure['uuid']
    file_name = brochure['file_name']
    file_path = brochures_file_path / f"{file_name}.pdf"

    # Upload the user provided file to OpenAI
    message_file = client.files.create(
    file=open(file_path, "rb"), purpose="assistants"
    )

    # Attach the file to the message for extraction
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": 
    """
    If a value is not explicitly mentioned, return 'N/A'. The output should be structured as JSON.\n\n
    Do not, under any circumstances, infer values by calculation or estimation, only include values that exist in the document and that you are confident of. Marking a value as 'N/A' is better than being wrong.\n 
    Only output the json content, no other messages are needed.\n
    Dollar values should have dollar signs next to them to be included, SF measurements should have SF units to be included.\n
    
    Extract the following details from the attached commercial real estate PDF and return JSON-formatted results:\n

    {
        sale_price: The total price of the property if it's for sale ($, $ per SF, $ per acre, etc.).\n
        lease_rate: The total lease rate of the property if it's for lease ($, $ per SF, $ per acre, market, etc.). Different than additional rent. Do not use operating costs or additional rent here.\n
        property_size: The total area of the property (SF, acres, etc.).\n
        year_built: The year the property was constructed.\n
        number_of_units: The number of rental units in the property.\n
        rental_income_estimate: The amount of rental income expected for the property.\n
        property_tax: The annual tax amount for the property.\n
        property_vacancy: The property's vacancy (number of units, SF vacant, percentage, etc.).\n
        num_dock_level_doors: The number of dock level doors in the property.\n
        dimensions_dock_level_doors: The dimensions of the dock level doors in the property.\n
        num_grade_level_doors: The number of grade level doors in the property.\n
        dimensions_grade_level_doors: The dimensions of the grade level doors in the property.\n
        ceiling_height: The ceiling height of the property.\n
        ceiling_clear_height: The clear height of the property.\n
        parking_spaces: The number of parking spaces on the property.\n
        year_built: The year the property was constructed.\n
        ti_allowance: The tenant improvement allowance for the property.\n
        number_of_units: The number of units in the property.\n
        operating_expenses: The total operating expenses for the property. Sometimes this is called additional rent.\n
    }

    If the pdf has details on more than one property (e.g. a portfolio), extract the details for each property separately. 
    An output with multiple properties should be structured in the following format: [{...}, {...}, ...] whereas a single 
    property should be structured as {...}.
    """,
                # Attach the file to the message.
                "attachments": [
                    {"file_id": message_file.id, "tools": [{"type": "file_search"}]}
                ],
            }
        ]
    )

    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id, assistant_id=assistant.id
    )

    messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

    message_content = re.sub(r'```json\n(.*?)```', r'\1', messages[0].content[0].text.value, flags=re.DOTALL)

    client.files.delete(message_file.id)
    processed_brochures_list.append({"uuid": uuid, "pdf": {"file_name": file_name, "content": message_content}})
    
    # Save the updated data to the JSON file
    with open(processed_brochures_file_path / output_json_file, 'w') as f:
        json.dump(processed_brochures_list, f, indent=2)

In [ ]:
print(processed_brochures_list)

[{'uuid': '34670172aa084f8e9ee12435a4241792', 'pdf': {'file_name': 'Westpoint_Business_Park_Brochure_18023', 'content': '[\n    {\n        "sale_price": "N/A",\n        "lease_rate": "Market",\n        "property_size": "13,704 sf",\n        "year_built": "N/A",\n        "number_of_units": "N/A",\n        "rental_income_estimate": "N/A",\n        "property_tax": "N/A",\n        "property_vacancy": "N/A",\n        "num_dock_level_doors": 3,\n        "dimensions_dock_level_doors": "10’ x 8’",\n        "num_grade_level_doors": "N/A",\n        "dimensions_grade_level_doors": "N/A",\n        "ceiling_height": "28’ clear",\n        "ceiling_clear_height": "N/A",\n        "parking_spaces": "N/A",\n        "ti_allowance": "N/A",\n        "operating_expenses": "$4.83 psf (2025) *plus mgmt. fee*"\n    }\n]\n'}}, {'uuid': '34670172aa084f8e9ee12435a4241792', 'pdf': {'file_name': 'Westpoint_Business_Park_Brochure_18125', 'content': '[\n    {\n        "sale_price": "N/A",\n        "lease_rate": "Mark

#### Post Processing

Data will be in the format: [ {uuid: "the_uuid", "pdf": {"file_name": "file_name.pdf", "content": "Text from open AI"} }, {uuid: "the_uuid", "pdf": {...}}, ... ]

For duplicate UUIDs, it means that there was more than one brochure for that property, so we need to check which duplicate has the most values not equal to "N/A"

Problem already solved:
- Sometimes two property listings have the same brochure url. Each of these properties will have an output from OpenAI for that brochure. 
- One potential problem with this is if The OpenAI output is by chance different for the same brochure input but hopefully the model is advanced enough to give consistent output for the same input in this simple problem.


In [ ]:
raise
input_dir = Path('/Volumes/James Mck/re_analysis/data/brochures/')
output_dir = Path('/Volumes/James Mck/re_analysis/data/unlabeled_train/')
PDFs = [f for f in os.listdir(input_dir) if not f.startswith("._") and f.endswith(".pdf")]

RuntimeError: No active exception to reraise

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
data = []

# Define the chunk size (number of iterations after which to save the JSON file)
chunk_size = 30

# Loop through the PDF files with a progress bar
for i, pdf_file in tqdm(enumerate(PDFs)):
    try:
        # Parse the PDF file to markdown
        parsed_PDF = pymupdf4llm.to_markdown(input_dir / pdf_file, show_progress=False)
        
        # Append the parsed text to the data list
        data.append({"text": parsed_PDF})
        
        # Check if the chunk size is reached
        if (i + 1) % chunk_size == 0:
            # Load existing data from the JSON file if it exists
            if (output_dir / 'edm_brochures_dataset.json').exists():
                with open(output_dir / 'edm_brochures_dataset.json', 'r') as f:
                    existing_data = json.load(f)
                # Append the new data to the existing data
                existing_data.extend(data)
                data = existing_data  # Reset data to the combined list
            
            # Save the updated data to the JSON file
            with open(output_dir / 'edm_brochures_dataset.json', 'w') as f:
                json.dump(data, f, indent=2)
            
            # Reset the data list for the next chunk
            data = []
    
    except Exception as e:
        print(f'Failed to parse: {pdf_file}. Error: {e}')
        continue

# Save any remaining data that didn't make up a full chunk
if data:
    if (output_dir / 'edm_brochures_dataset.json').exists():
        with open(output_dir / 'edm_brochures_dataset.json', 'r') as f:
            existing_data = json.load(f)
        existing_data.extend(data)
        data = existing_data
    
    with open(output_dir / 'edm_brochures_dataset.json', 'w') as f:
        json.dump(data, f, indent=2)

42it [06:27,  3.92s/it]

MuPDF error: system error: read error: Input/output error

MuPDF error: system error: read error: Input/output error

MuPDF error: system error: read error: Input/output error

MuPDF error: system error: read error: Input/output error

MuPDF error: system error: read error: Input/output error

MuPDF error: system error: read error: Input/output error

MuPDF error: system error: read error: Input/output error

MuPDF error: library error: jpeg error: Not a JPEG file: starts with 0xff 0xd9

MuPDF error: system error: read error: Input/output error

MuPDF error: system error: read error: Input/output error

MuPDF error: library error: jpeg error: Not a JPEG file: starts with 0xff 0xd9

MuPDF error: system error: read error: Input/output error

MuPDF error: system error: read error: Input/output error

MuPDF error: library error: jpeg error: Not a JPEG file: starts with 0xff 0xd9

MuPDF error: system error: read error: Input/output error

MuPDF error: system error: read error: Input/output 

244it [44:12,  4.53s/it]